# TorchVision Tutorial Based Pipeline

Following TORCHVISION OBJECT DETECTION FINETUNING TUTORIAL at https://pytorch.org/tutorials/intermediate/torchvision_tutorial.html

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
from pathlib import Path
import os
# Vlad's drive
# repo_path = Path.cwd()/'drive/MyDrive/calcification-detection-project/calcification_detecion/calc-det'
# Joaquin's drive
repo_path = Path.cwd()/'drive/MyDrive/calcification_detection/calc-det/notebooks'
os.chdir(str(repo_path))

thispath = Path.cwd().resolve()
import sys; sys.path.insert(0, str(thispath.parent))

Copy extracted patches used for detection (224 size with stride 100) to the local machine. ***Needs to be done once per machine instantization***. 

~ 3 minutes to complete

In [ ]:
# # Vlad's Drive
# # !cp -r /content/drive/MyDrive/calcification-detection-project/calcification_detecion/new_data_rois/data_rois.zip /home/
# # Joaquin's Drive
# !cp -r /content/drive/MyDrive/calcification_detection/rois_all_method/data_rois_224.zip /home/data_rois.zip
# !unzip /home/data_rois.zip -d /home
# !mv /home/home/vzalevskyi/projects/data_rois /home/data_rois
# !rm -r /home/home
# !pip install transformers

Now let's instantiate the model and the optimizer

And now let's train the model for 10 epochs, evaluating at the end of every epoch.

In [3]:
cfg = {
    'model': {
        'activation': 'LeakyReLU',
        'bloc_act': None,
        'use_middle_activation': True,
        'dropout': 0.4,
        'fc_dims': None,
        'freeze_weights': False,
        'backbone': 'net2',
        'pretrained': True,
        'checkpoint_path': None,
        'anchor_sizes': [12, 14],
        'anchor_ratios': [1.0],
        'num_classes': 2
    },
    'dataset': {
        'extract_patches': False,
        'delete_previous': False,
        'extract_patches_method': 'all',
        'patch_size': 224,
        'stride': 100,
        'min_breast_fraction_roi': 0.7,
        'n_jobs': -1,
        'cropped_imgs': True,
        'ignore_diameter_px': 15,
        'train_neg_to_pos_ratio': None,
        'balancing_seed': 0,
        'normalization': 'min_max',
        'get_lesion_bboxes': True,
        'for_detection_net': True,
        'patch_images_path': '/home/data_rois/',
        'detection_bbox_size':14
    },
    'dataloaders': {
        'train_batch_size': 8,
        'val_batch_size': 16
    },
    'data_aug': {
        'prob': 0
    },
    'training': {
        'criterion': 'BCEWithLogitsLoss',
        'optimizer': 'Adam',
        'optimizer_args': {
            'lr': 0.0001 #, 'momentum': 0.9
        },
        'lr_scheduler': 'StepLR',
        'lr_scheduler_args': {
            'step_size': 3, 'gamma': 0.1
        },
        'n_epochs': 30,
        'best_metric': 'f1_score',
        'resume_training': False,
        'early_stopping': True,
        'early_stopping_args':{
            'min_diff': 0.0001,
            'max_epoch': 3
        },
        'log_iters': 100,
        'max_iters_per_epoch': None
        },
    'experiment_name': '32_resnet_01'
}

import yaml

cfg_path = str(thispath.parent.parent/'calc-det/deep_learning/detection_models/config.yml')
with open(cfg_path, 'w') as yaml_file:
    yaml.dump(cfg, yaml_file, default_flow_style=False)

In [4]:
from pathlib import Path
thispath = Path.cwd().resolve()
import sys; sys.path.insert(0, str(thispath.parent))

import logging
import torch
import torchvision
import yaml

import torch.nn as nn
import torch.optim as optim

from torch.optim import lr_scheduler
from torch.utils.data import DataLoader
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator

import deep_learning.dl_utils as dl_utils
from deep_learning.dataset.dataset import INBreast_Dataset_pytorch
# from deep_learning.classification_models.base_classifier import CNNClasssifier
from deep_learning.models.base_classifier import CNNClasssifier
from deep_learning.detection_models.train import train_model

logging.basicConfig(level=logging.INFO)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


def main():
    # read the configuration file
    config_path = str(thispath.parent.parent/'calc-det/deep_learning/detection_models/config.yml')
    with open(config_path, "r") as ymlfile:
        cfg = yaml.safe_load(ymlfile)

    # for colab it needs to be specified additionally
    dataset_arguments = cfg['dataset']

    # use the configuration for the dataset
    dataset_arguments = cfg['dataset']
    dataset_arguments['patch_images_path'] = Path(dataset_arguments['patch_images_path'])
    datasets = {
        'train': INBreast_Dataset_pytorch(
            partitions=['train'], neg_to_pos_ratio=dataset_arguments['train_neg_to_pos_ratio'],
            **dataset_arguments),
        'val': INBreast_Dataset_pytorch(
            partitions=['validation'], neg_to_pos_ratio=None, **dataset_arguments)
    }

    # use the configuration for the dataloaders
    def collate_fn(batch):
        return tuple(zip(*batch))
    dataloaders = {
        'val': DataLoader(
            datasets['val'], batch_size=cfg['dataloaders']['val_batch_size'],
            num_workers=4, collate_fn=collate_fn, drop_last=False),
        'train': DataLoader(
            datasets['train'], batch_size=cfg['dataloaders']['train_batch_size'],
            shuffle=True, collate_fn=collate_fn, num_workers=4, drop_last=False)
    }

    # model settings
    if cfg['model']['checkpoint_path'] is not None:
        model_ckpt = torch.load(cfg['model']['checkpoint_path'])
        model = dl_utils.get_model_from_checkpoint(model_ckpt, cfg['model']['freeze_weights'])
    else:
        model = CNNClasssifier(
            activation=getattr(nn, cfg['model']['activation'])(),
            dropout=cfg['model']['dropout'],
            fc_dims=cfg['model']['fc_dims'],
            freeze_weights=cfg['model']['freeze_weights'],
            backbone=cfg['model']['backbone'],
            pretrained=cfg['model']['pretrained'],
        )
        model = model.model

    modules = list(model.children())[:-2]      # delete the last fc layers.
    last_submodule_childs = list(modules[-1][-1].children())
    for i in range(len(last_submodule_childs)-1, -1, -1):
        if hasattr(last_submodule_childs[i], 'out_channels'):
            out_channels = last_submodule_childs[i].out_channels
            break
    model_backbone = nn.Sequential(*modules)
    model_backbone.out_channels = out_channels
    anchor_generator = AnchorGenerator(
        sizes=(cfg['model']['anchor_sizes'],),
        aspect_ratios=(cfg['model']['anchor_ratios'],))

    roi_pooler = torchvision.ops.MultiScaleRoIAlign(
        featmap_names=['0'], output_size=7, sampling_ratio=2)

    # put the pieces together inside a FasterRCNN model
    model = FasterRCNN(
        model_backbone,
        num_classes=cfg['model']['num_classes'],
        rpn_anchor_generator=anchor_generator,
        box_roi_pool=roi_pooler,
        image_mean=[0., 0., 0.],
        image_std=[1., 1., 1.],
    )

    model.to(device)

    # training configs
    optimizer = getattr(optim, cfg['training']['optimizer'])
    optimizer = optimizer(model.parameters(), **cfg['training']['optimizer_args'])

    scheduler = getattr(lr_scheduler, cfg['training']['lr_scheduler'])
    scheduler = scheduler(optimizer, **cfg['training']['lr_scheduler_args'])

    # train the model
    train_model(datasets, dataloaders, model, optimizer, scheduler, cfg)

In [ ]:
# # cfg['model']['activation'] = 'LeakyReLU'
# # cfg['model']['dropout'] = 0.4
# # cfg['model']['fc_dims'] = None
# cfg['model']['backbone'] = 'resnet50'
# cfg['model']['pretrained'] = True
# cfg['model']['checkpoint_path'] = '/content/drive/MyDrive/drive_vlad/deepl_runs/resnet50_05/resnet50_05.pt'
# cfg['model']['anchor_sizes'] = [14]
# cfg['model']['anchor_ratios'] = [1.0]
# cfg['model']['num_classes'] = 2

# cfg['dataset']['train_neg_to_pos_ratio'] = None
# cfg['dataset']['normalization'] = 'z_score'

# cfg['dataloaders']['train_batch_size'] = 6
# cfg['dataloaders']['val_batch_size'] = 12

# cfg['training']['optimizer'] = 'Adam'
# cfg['training']['optimizer_args'] = {'lr': 0.0001}
# cfg['training']['lr_scheduler'] = 'StepLR'
# cfg['training']['lr_scheduler_args'] = {'step_size': 3, 'gamma': 0.1}
# cfg['training']['n_epochs'] = 15
# cfg['training']['early_stopping'] = False
# cfg['training']['early_stopping_args'] = {'min_diff': 0.0001, 'max_epoch': 3}
# cfg['training']['log_iters'] = 100
# cfg['training']['best_metric'] = 'AP_IoU_0.50_all'
# cfg['training']['max_iters_per_epoch'] = 200

# cfg['experiment_name'] = 'det_resnet50_our_02'

# cfg_path = str(thispath.parent.parent/'calc-det/deep_learning/detection_models/config.yml')
# with open(cfg_path, 'w') as yaml_file:
#     yaml.dump(cfg, yaml_file, default_flow_style=False)
# main()

In [ ]:

# # cfg['model']['activation'] = 'LeakyReLU'
# # cfg['model']['dropout'] = 0.4
# # cfg['model']['fc_dims'] = None
# cfg['model']['backbone'] = 'resnet50'
# cfg['model']['pretrained'] = True
# cfg['model']['checkpoint_path'] = '/content/drive/MyDrive/drive_vlad/deepl_runs/resnet50_05/resnet50_05.pt'
# cfg['model']['anchor_sizes'] = [12, 14, 16]
# cfg['model']['anchor_ratios'] = [1.0]
# cfg['model']['num_classes'] = 2

# cfg['dataset']['train_neg_to_pos_ratio'] = None
# cfg['dataset']['normalization'] = 'z_score'

# cfg['dataloaders']['train_batch_size'] = 6
# cfg['dataloaders']['val_batch_size'] = 12

# cfg['training']['optimizer'] = 'Adam'
# cfg['training']['optimizer_args'] = {'lr': 0.0001}
# cfg['training']['lr_scheduler'] = 'StepLR'
# cfg['training']['lr_scheduler_args'] = {'step_size': 3, 'gamma': 0.1}
# cfg['training']['n_epochs'] = 15
# cfg['training']['early_stopping'] = False
# cfg['training']['early_stopping_args'] = {'min_diff': 0.0001, 'max_epoch': 3}
# cfg['training']['log_iters'] = 100
# cfg['training']['best_metric'] = 'AP_IoU_0.50_all'
# cfg['training']['max_iters_per_epoch'] = 200

# cfg['experiment_name'] = 'det_resnet50_our_02'

# cfg_path = str(thispath.parent.parent/'calc-det/deep_learning/detection_models/config.yml')
# with open(cfg_path, 'w') as yaml_file:
#     yaml.dump(cfg, yaml_file, default_flow_style=False)
# main()

In [ ]:
# # cfg['model']['activation'] = 'LeakyReLU'
# # cfg['model']['dropout'] = 0.4
# # cfg['model']['fc_dims'] = None
# cfg['model']['backbone'] = 'resnet50'
# cfg['model']['pretrained'] = True
# cfg['model']['checkpoint_path'] = '/content/drive/MyDrive/drive_vlad/deepl_runs/resnet50_05/resnet50_05.pt'
# cfg['model']['anchor_sizes'] = [14]
# cfg['model']['anchor_ratios'] = [1.0]
# cfg['model']['num_classes'] = 2

# cfg['dataset']['train_neg_to_pos_ratio'] = None
# cfg['dataset']['normalization'] = 'z_score'

# cfg['dataloaders']['train_batch_size'] = 6
# cfg['dataloaders']['val_batch_size'] = 12

# cfg['training']['optimizer'] = 'Adam'
# cfg['training']['optimizer_args'] = {'lr': 0.0001}
# cfg['training']['lr_scheduler'] = 'StepLR'
# cfg['training']['lr_scheduler_args'] = {'step_size': 3, 'gamma': 0.1}
# cfg['training']['n_epochs'] = 15
# cfg['training']['early_stopping'] = False
# cfg['training']['early_stopping_args'] = {'min_diff': 0.0001, 'max_epoch': 3}
# cfg['training']['log_iters'] = 100
# cfg['training']['best_metric'] = 'AP_IoU_0.50_all'
# cfg['training']['max_iters_per_epoch'] = None

# cfg['experiment_name'] = 'resnet50_00'

# cfg_path = str(thispath.parent.parent/'calc-det/deep_learning/detection_models/config.yml')
# with open(cfg_path, 'w') as yaml_file:
#     yaml.dump(cfg, yaml_file, default_flow_style=False)
# main()

/content/drive/MyDrive/calcification_detection/calc-det/deep_learning/dataset/dataset.py:52: DtypeWarning: Columns (16,17) have mixed types.Specify dtype option on import or set low_memory=False.
  return_lesions_mask=get_lesion_bboxes, max_lesion_diam_mm=None, use_muscle_mask=False
INFO:root:Storing experiment in: /content/drive/MyDrive/calcification_detection/data/deepl_runs/resnet50_00


creating index...
index created!
Epoch: [0]  [  0/729]  eta: 0:18:29  lr: 0.000000  loss: 1.4622 (1.4622)  loss_classifier: 0.7302 (0.7302)  loss_box_reg: 0.0014 (0.0014)  loss_objectness: 0.6497 (0.6497)  loss_rpn_box_reg: 0.0809 (0.0809)  time: 1.5218  data: 0.4650  max mem: 9834
Epoch: [0]  [100/729]  eta: 0:10:39  lr: 0.000014  loss: 0.1023 (0.3378)  loss_classifier: 0.0307 (0.1245)  loss_box_reg: 0.0075 (0.0090)  loss_objectness: 0.0509 (0.1699)  loss_rpn_box_reg: 0.0167 (0.0344)  time: 1.0087  data: 0.0208  max mem: 11723
Epoch: [0]  [200/729]  eta: 0:08:55  lr: 0.000028  loss: 0.0892 (0.2315)  loss_classifier: 0.0266 (0.0808)  loss_box_reg: 0.0230 (0.0149)  loss_objectness: 0.0286 (0.1095)  loss_rpn_box_reg: 0.0091 (0.0263)  time: 1.0105  data: 0.0226  max mem: 11723
Epoch: [0]  [300/729]  eta: 0:07:14  lr: 0.000041  loss: 0.0717 (0.1900)  loss_classifier: 0.0205 (0.0644)  loss_box_reg: 0.0231 (0.0200)  loss_objectness: 0.0214 (0.0837)  loss_rpn_box_reg: 0.0078 (0.0218)  time: 1

INFO:root:Train Loss: 0.1269, AP_IoU_0.50_all: 0.7247AR_IoU_0.50_0.95_all_mdets_100: 0.5442947998503553


DONE (t=0.15s).
IoU metric: bbox
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.410
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.725
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.441
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.410
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.243
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.478
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.544
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.544
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = -1.000
Epo

INFO:root:Train Loss: 0.0594, AP_IoU_0.50_all: 0.8295AR_IoU_0.50_0.95_all_mdets_100: 0.6292181069958848


...

INFO:root:Train Loss: 0.0125, AP_IoU_0.50_all: 0.9131AR_IoU_0.50_0.95_all_mdets_100: 0.7780022446689113


DONE (t=0.10s).
IoU metric: bbox
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.709
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.913
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.853
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.709
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.318
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.679
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.778
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.778
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = -1.000
Epo

INFO:root:Train Loss: 0.0125, AP_IoU_0.50_all: 0.9131AR_IoU_0.50_0.95_all_mdets_100: 0.7773662551440329


DONE (t=0.12s).
IoU metric: bbox
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.708
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.913
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.853
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.708
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.318
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.677
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.777
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.777
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = -1.000


INFO:root:----------
INFO:root:Training complete in 202m 56s
INFO:root:Best val AP_IoU_0.50_all/val: 0.913054, AR_IoU_0.50_0.95_all: 0.777366, epoch 13


In [ ]:
# # cfg['model']['activation'] = 'LeakyReLU'
# # cfg['model']['dropout'] = 0.4
# # cfg['model']['fc_dims'] = None
# cfg['model']['backbone'] = 'resnet50'
# cfg['model']['pretrained'] = True
# cfg['model']['checkpoint_path'] = '/content/drive/MyDrive/drive_vlad/deepl_runs/resnet50_05/resnet50_05.pt'
# cfg['model']['anchor_sizes'] = [14]
# cfg['model']['anchor_ratios'] = [1.0]
# cfg['model']['num_classes'] = 2
# cfg['model']['freeze_weights'] = True

# cfg['dataset']['train_neg_to_pos_ratio'] = None
# cfg['dataset']['normalization'] = 'z_score'

# cfg['dataloaders']['train_batch_size'] = 6
# cfg['dataloaders']['val_batch_size'] = 12

# cfg['training']['optimizer'] = 'Adam'
# cfg['training']['optimizer_args'] = {'lr': 0.0001}
# cfg['training']['lr_scheduler'] = 'StepLR'
# cfg['training']['lr_scheduler_args'] = {'step_size': 3, 'gamma': 0.1}
# cfg['training']['n_epochs'] = 10
# cfg['training']['early_stopping'] = False
# cfg['training']['early_stopping_args'] = {'min_diff': 0.0001, 'max_epoch': 3}
# cfg['training']['log_iters'] = 100
# cfg['training']['best_metric'] = 'AP_IoU_0.50_all'
# cfg['training']['max_iters_per_epoch'] = None


# cfg['experiment_name'] = 'resnet50_01'

# cfg_path = str(thispath.parent.parent/'calc-det/deep_learning/detection_models/config.yml')
# with open(cfg_path, 'w') as yaml_file:
#     yaml.dump(cfg, yaml_file, default_flow_style=False)
# main()

/content/drive/MyDrive/calcification_detection/calc-det/deep_learning/dataset/dataset.py:52: DtypeWarning: Columns (16,17) have mixed types.Specify dtype option on import or set low_memory=False.
  return_lesions_mask=get_lesion_bboxes, max_lesion_diam_mm=None, use_muscle_mask=False
INFO:root:Storing experiment in: /content/drive/MyDrive/calcification_detection/data/deepl_runs/resnet50_01


creating index...
index created!
Epoch: [0]  [  0/729]  eta: 0:11:53  lr: 0.000000  loss: 1.4761 (1.4761)  loss_classifier: 0.6682 (0.6682)  loss_box_reg: 0.0011 (0.0011)  loss_objectness: 0.7191 (0.7191)  loss_rpn_box_reg: 0.0876 (0.0876)  time: 0.9791  data: 0.5178  max mem: 11723
Epoch: [0]  [100/729]  eta: 0:05:01  lr: 0.000014  loss: 0.1426 (0.3851)  loss_classifier: 0.0457 (0.1311)  loss_box_reg: 0.0090 (0.0064)  loss_objectness: 0.0760 (0.2095)  loss_rpn_box_reg: 0.0176 (0.0380)  time: 0.4684  data: 0.0234  max mem: 11723
Epoch: [0]  [200/729]  eta: 0:04:09  lr: 0.000028  loss: 0.1310 (0.2780)  loss_classifier: 0.0399 (0.0900)  loss_box_reg: 0.0131 (0.0104)  loss_objectness: 0.0611 (0.1458)  loss_rpn_box_reg: 0.0164 (0.0318)  time: 0.4642  data: 0.0215  max mem: 11723
Epoch: [0]  [300/729]  eta: 0:03:21  lr: 0.000041  loss: 0.1132 (0.2368)  loss_classifier: 0.0372 (0.0756)  loss_box_reg: 0.0127 (0.0123)  loss_objectness: 0.0529 (0.1204)  loss_rpn_box_reg: 0.0165 (0.0286)  time: 

INFO:root:Train Loss: 0.1831, AP_IoU_0.50_all: 0.1212AR_IoU_0.50_0.95_all_mdets_100: 0.15387205387205388


DONE (t=0.18s).
IoU metric: bbox
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.034
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.121
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.008
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.034
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.057
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.121
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.154
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.154
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = -1.000
Epo

INFO:root:Train Loss: 0.1339, AP_IoU_0.50_all: 0.2453AR_IoU_0.50_0.95_all_mdets_100: 0.2625888514777404


...

INFO:root:Train Loss: 0.1059, AP_IoU_0.50_all: 0.3715AR_IoU_0.50_0.95_all_mdets_100: 0.3808080808080808


DONE (t=0.22s).
IoU metric: bbox
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.176
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.372
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.133
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.176
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.170
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.311
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.381
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.381
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = -1.000
Epo

INFO:root:Train Loss: 0.1055, AP_IoU_0.50_all: 0.3699AR_IoU_0.50_0.95_all_mdets_100: 0.3761690983913206


DONE (t=0.20s).
IoU metric: bbox
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.175
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.370
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.134
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.175
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.167
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.308
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.376
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.376
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = -1.000


INFO:root:----------
INFO:root:Training complete in 69m 17s
INFO:root:Best val AP_IoU_0.50_all/val: 0.369903, AR_IoU_0.50_0.95_all: 0.376169, epoch 7


In [ ]:
# cfg['model']['activation'] = 'LeakyReLU'
# cfg['model']['dropout'] = 0.4
# cfg['model']['fc_dims'] = None
cfg['model']['backbone'] = 'resnet50'
cfg['model']['pretrained'] = False
cfg['model']['checkpoint_path'] = None
cfg['model']['anchor_sizes'] = [14]
cfg['model']['anchor_ratios'] = [1.0]
cfg['model']['num_classes'] = 2
cfg['model']['freeze_weights'] = False

cfg['dataset']['train_neg_to_pos_ratio'] = None
cfg['dataset']['normalization'] = 'z_score'

cfg['dataloaders']['train_batch_size'] = 6
cfg['dataloaders']['val_batch_size'] = 12

cfg['training']['optimizer'] = 'Adam'
cfg['training']['optimizer_args'] = {'lr': 0.0001}
cfg['training']['lr_scheduler'] = 'StepLR'
cfg['training']['lr_scheduler_args'] = {'step_size': 3, 'gamma': 0.1}
cfg['training']['n_epochs'] = 20
cfg['training']['early_stopping'] = False
cfg['training']['early_stopping_args'] = {'min_diff': 0.0001, 'max_epoch': 3}
cfg['training']['log_iters'] = 100
cfg['training']['best_metric'] = 'AP_IoU_0.50_all'
cfg['training']['max_iters_per_epoch'] = None

cfg['experiment_name'] = 'd_resnet50_02'

cfg_path = str(thispath.parent.parent/'calc-det/deep_learning/detection_models/config.yml')
with open(cfg_path, 'w') as yaml_file:
    yaml.dump(cfg, yaml_file, default_flow_style=False)
main()

/content/drive/MyDrive/calcification_detection/calc-det/deep_learning/dataset/dataset.py:52: DtypeWarning: Columns (16,17) have mixed types.Specify dtype option on import or set low_memory=False.
  return_lesions_mask=get_lesion_bboxes, max_lesion_diam_mm=None, use_muscle_mask=False
INFO:root:Storing experiment in: /content/drive/MyDrive/calcification_detection/data/deepl_runs/d_resnet50_02


creating index...
index created!
Epoch: [0]  [  0/729]  eta: 0:41:52  lr: 0.000000  loss: 1.7249 (1.7249)  loss_classifier: 0.6844 (0.6844)  loss_box_reg: 0.0030 (0.0030)  loss_objectness: 0.9173 (0.9173)  loss_rpn_box_reg: 0.1203 (0.1203)  time: 3.4466  data: 0.3955  max mem: 9737
Epoch: [0]  [100/729]  eta: 0:19:53  lr: 0.000014  loss: 0.1296 (0.3746)  loss_classifier: 0.0452 (0.1119)  loss_box_reg: 0.0087 (0.0070)  loss_objectness: 0.0647 (0.2063)  loss_rpn_box_reg: 0.0237 (0.0494)  time: 1.8916  data: 0.0160  max mem: 11630
Epoch: [0]  [200/729]  eta: 0:16:39  lr: 0.000028  loss: 0.1222 (0.2736)  loss_classifier: 0.0376 (0.0799)  loss_box_reg: 0.0113 (0.0097)  loss_objectness: 0.0574 (0.1420)  loss_rpn_box_reg: 0.0226 (0.0421)  time: 1.8823  data: 0.0153  max mem: 11630
Epoch: [0]  [300/729]  eta: 0:13:29  lr: 0.000041  loss: 0.1173 (0.2358)  loss_classifier: 0.0360 (0.0691)  loss_box_reg: 0.0112 (0.0120)  loss_objectness: 0.0498 (0.1170)  loss_rpn_box_reg: 0.0164 (0.0377)  time: 1

INFO:root:Train Loss: 0.1816, AP_IoU_0.50_all: 0.4052AR_IoU_0.50_0.95_all_mdets_100: 0.3002618780396558


DONE (t=0.16s).
IoU metric: bbox
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.140
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.405
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.039
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.140
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.125
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.252
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.300
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.300
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = -1.000
Epo

INFO:root:Train Loss: 0.1072, AP_IoU_0.50_all: 0.5961AR_IoU_0.50_0.95_all_mdets_100: 0.5132435465768799


...

INFO:root:Train Loss: 0.0484, AP_IoU_0.50_all: 0.7492AR_IoU_0.50_0.95_all_mdets_100: 0.6533857089412646


DONE (t=0.14s).
IoU metric: bbox
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.497
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.749
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.585
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.497
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.277
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.563
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.653
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.653
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = -1.000
Epo

INFO:root:Train Loss: 0.0484, AP_IoU_0.50_all: 0.7563AR_IoU_0.50_0.95_all_mdets_100: 0.6581369248035915


DONE (t=0.15s).
IoU metric: bbox
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.501
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.756
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.596
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.501
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.279
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.566
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.658
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.658
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = -1.000
Epo

KeyboardInterrupt: ignored

In [ ]:
# a = torch.load('/content/drive/MyDrive/calcification_detection/data/deepl_runs/d_resnet50_02/d_resnet50_02_AP_IoU_0.50_all.pt')

In [ ]:
# cfg['model']['activation'] = 'LeakyReLU'
# cfg['model']['dropout'] = 0.4
# cfg['model']['fc_dims'] = None
cfg['model']['backbone'] = 'efficientnet_b3'
cfg['model']['pretrained'] = False
cfg['model']['checkpoint_path'] = '/content/drive/MyDrive/drive_vlad/deepl_runs/efficientnet_b3_02/efficientnet_b3_02.pt'
cfg['model']['anchor_sizes'] = [14]
cfg['model']['anchor_ratios'] = [1.0]
cfg['model']['num_classes'] = 2
cfg['model']['freeze_weights'] = False

cfg['dataset']['train_neg_to_pos_ratio'] = None
cfg['dataset']['normalization'] = 'z_score'

cfg['dataloaders']['train_batch_size'] = 4
cfg['dataloaders']['val_batch_size'] = 8

cfg['training']['optimizer'] = 'Adam'
cfg['training']['optimizer_args'] = {'lr': 0.0001}
cfg['training']['lr_scheduler'] = 'StepLR'
cfg['training']['lr_scheduler_args'] = {'step_size': 3, 'gamma': 0.1}
cfg['training']['n_epochs'] = 15
cfg['training']['early_stopping'] = False
cfg['training']['early_stopping_args'] = {'min_diff': 0.0001, 'max_epoch': 3}
cfg['training']['log_iters'] = 100
cfg['training']['best_metric'] = 'AP_IoU_0.50_all'
cfg['training']['max_iters_per_epoch'] = None

cfg['experiment_name'] = 'd_efficientnet_b3_00'

cfg_path = str(thispath.parent.parent/'calc-det/deep_learning/detection_models/config.yml')
with open(cfg_path, 'w') as yaml_file:
    yaml.dump(cfg, yaml_file, default_flow_style=False)
main()

/content/drive/MyDrive/calcification_detection/calc-det/deep_learning/dataset/dataset.py:52: DtypeWarning: Columns (16,17) have mixed types.Specify dtype option on import or set low_memory=False.
  return_lesions_mask=get_lesion_bboxes, max_lesion_diam_mm=None, use_muscle_mask=False
Downloading: "https://download.pytorch.org/models/efficientnet_b3_rwightman-cf984f9c.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b3_rwightman-cf984f9c.pth


  0%|          | 0.00/47.2M [00:00<?, ?B/s]

INFO:root:Storing experiment in: /content/drive/MyDrive/calcification_detection/data/deepl_runs/d_efficientnet_b3_00


creating index...
index created!
Epoch: [0]  [   0/1093]  eta: 1:32:41  lr: 0.000000  loss: 1.4644 (1.4644)  loss_classifier: 0.7059 (0.7059)  loss_box_reg: 0.0012 (0.0012)  loss_objectness: 0.7059 (0.7059)  loss_rpn_box_reg: 0.0514 (0.0514)  time: 5.0884  data: 0.2398  max mem: 10361
Epoch: [0]  [ 100/1093]  eta: 0:11:25  lr: 0.000010  loss: 0.4939 (0.8937)  loss_classifier: 0.1325 (0.3467)  loss_box_reg: 0.0019 (0.0030)  loss_objectness: 0.3349 (0.4963)  loss_rpn_box_reg: 0.0319 (0.0477)  time: 0.6463  data: 0.0098  max mem: 11615
Epoch: [0]  [ 200/1093]  eta: 0:09:56  lr: 0.000020  loss: 0.1934 (0.6072)  loss_classifier: 0.0390 (0.2082)  loss_box_reg: 0.0066 (0.0049)  loss_objectness: 0.1329 (0.3533)  loss_rpn_box_reg: 0.0223 (0.0408)  time: 0.6469  data: 0.0117  max mem: 11615
Epoch: [0]  [ 300/1093]  eta: 0:08:43  lr: 0.000030  loss: 0.1238 (0.4660)  loss_classifier: 0.0277 (0.1530)  loss_box_reg: 0.0138 (0.0078)  loss_objectness: 0.0601 (0.2696)  loss_rpn_box_reg: 0.0144 (0.0356)

INFO:root:Train Loss: 0.2142, AP_IoU_0.50_all: 0.5738AR_IoU_0.50_0.95_all_mdets_100: 0.3912832023943135


...

INFO:root:Train Loss: 0.0356, AP_IoU_0.50_all: 0.7867AR_IoU_0.50_0.95_all_mdets_100: 0.6128694350916573


DONE (t=0.32s).
IoU metric: bbox
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.518
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.787
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.595
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.518
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.274
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.548
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.613
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.613
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = -1.000
Epo

INFO:root:Train Loss: 0.0358, AP_IoU_0.50_all: 0.8210AR_IoU_0.50_0.95_all_mdets_100: 0.6404788627010849


DONE (t=0.14s).
IoU metric: bbox
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.537
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.821
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.616
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.537
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.283
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.567
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.640
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.640
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = -1.000


INFO:root:----------
INFO:root:Training complete in 190m 32s
INFO:root:Best val AP_IoU_0.50_all/val: 0.821040, AR_IoU_0.50_0.95_all: 0.640479, epoch 12


In [ ]:
# cfg['model']['activation'] = 'LeakyReLU'
# cfg['model']['dropout'] = 0.4
# cfg['model']['fc_dims'] = None
cfg['model']['backbone'] = 'efficientnet_b0'
cfg['model']['pretrained'] = True
cfg['model']['checkpoint_path'] = '/content/drive/MyDrive/drive_vlad/deepl_runs/efficientnet_b0_01/efficientnet_b0_01.pt'
cfg['model']['anchor_sizes'] = [14]
cfg['model']['anchor_ratios'] = [1.0]
cfg['model']['num_classes'] = 2
cfg['model']['freeze_weights'] = False

cfg['dataset']['train_neg_to_pos_ratio'] = None
cfg['dataset']['normalization'] = 'z_score'

cfg['dataloaders']['train_batch_size'] = 4
cfg['dataloaders']['val_batch_size'] = 8

cfg['training']['optimizer'] = 'Adam'
cfg['training']['optimizer_args'] = {'lr': 0.0001}
cfg['training']['lr_scheduler'] = 'StepLR'
cfg['training']['lr_scheduler_args'] = {'step_size': 3, 'gamma': 0.1}
cfg['training']['n_epochs'] = 15
cfg['training']['early_stopping'] = False
cfg['training']['early_stopping_args'] = {'min_diff': 0.0001, 'max_epoch': 3}
cfg['training']['log_iters'] = 100
cfg['training']['best_metric'] = 'AP_IoU_0.50_all'
cfg['training']['max_iters_per_epoch'] = None

cfg['experiment_name'] = 'd_efficientnet_b3_01'

cfg_path = str(thispath.parent.parent/'calc-det/deep_learning/detection_models/config.yml')
with open(cfg_path, 'w') as yaml_file:
    yaml.dump(cfg, yaml_file, default_flow_style=False)
main()

/content/drive/MyDrive/calcification_detection/calc-det/deep_learning/dataset/dataset.py:52: DtypeWarning: Columns (16,17) have mixed types.Specify dtype option on import or set low_memory=False.
  return_lesions_mask=get_lesion_bboxes, max_lesion_diam_mm=None, use_muscle_mask=False
INFO:root:Storing experiment in: /content/drive/MyDrive/calcification_detection/data/deepl_runs/d_efficientnet_b3_01


creating index...
index created!
Epoch: [0]  [   0/1093]  eta: 0:31:24  lr: 0.000000  loss: 1.4945 (1.4945)  loss_classifier: 0.7102 (0.7102)  loss_box_reg: 0.0010 (0.0010)  loss_objectness: 0.7334 (0.7334)  loss_rpn_box_reg: 0.0499 (0.0499)  time: 1.7240  data: 0.2950  max mem: 5920
Epoch: [0]  [ 100/1093]  eta: 0:07:15  lr: 0.000010  loss: 0.5087 (0.9020)  loss_classifier: 0.1459 (0.3457)  loss_box_reg: 0.0027 (0.0020)  loss_objectness: 0.3453 (0.5079)  loss_rpn_box_reg: 0.0327 (0.0464)  time: 0.4253  data: 0.0104  max mem: 6877
Epoch: [0]  [ 200/1093]  eta: 0:06:25  lr: 0.000020  loss: 0.1970 (0.6150)  loss_classifier: 0.0390 (0.2098)  loss_box_reg: 0.0042 (0.0037)  loss_objectness: 0.1311 (0.3616)  loss_rpn_box_reg: 0.0223 (0.0400)  time: 0.4252  data: 0.0107  max mem: 6877
Epoch: [0]  [ 300/1093]  eta: 0:05:40  lr: 0.000030  loss: 0.1163 (0.4685)  loss_classifier: 0.0272 (0.1544)  loss_box_reg: 0.0081 (0.0056)  loss_objectness: 0.0603 (0.2732)  loss_rpn_box_reg: 0.0165 (0.0353)  t

INFO:root:Train Loss: 0.2142, AP_IoU_0.50_all: 0.5370AR_IoU_0.50_0.95_all_mdets_100: 0.34216236438458664


DONE (t=0.20s).
IoU metric: bbox
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.202
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.537
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.089
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.202
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.163
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.285
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.342
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.342
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = -1.000
Epo

INFO:root:Train Loss: 0.0867, AP_IoU_0.50_all: 0.6793AR_IoU_0.50_0.95_all_mdets_100: 0.5119341563786008


...

INFO:root:Train Loss: 0.0393, AP_IoU_0.50_all: 0.8050AR_IoU_0.50_0.95_all_mdets_100: 0.6390946502057614


DONE (t=0.11s).
IoU metric: bbox
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.530
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.805
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.610
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.530
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.285
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.562
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.639
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.639
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = -1.000
Epo

INFO:root:Train Loss: 0.0396, AP_IoU_0.50_all: 0.8132AR_IoU_0.50_0.95_all_mdets_100: 0.6447437336326225


DONE (t=0.10s).
IoU metric: bbox
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.538
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.813
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.626
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.538
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.288
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.563
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.645
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.645
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = -1.000


INFO:root:----------
INFO:root:Training complete in 126m 30s
INFO:root:Best val AP_IoU_0.50_all/val: 0.813217, AR_IoU_0.50_0.95_all: 0.644744, epoch 12


In [ ]:
cfg['model']['backbone'] = 'resnet50'
cfg['model']['pretrained'] = True
cfg['model']['checkpoint_path'] = '/content/drive/MyDrive/drive_vlad/deepl_runs/resnet50_05/resnet50_05.pt'
cfg['model']['anchor_sizes'] = [7, 14]
cfg['model']['anchor_ratios'] = [1.0]
cfg['model']['num_classes'] = 2
cfg['model']['freeze_weights'] = False

cfg['dataset']['train_neg_to_pos_ratio'] = None
cfg['dataset']['normalization'] = 'z_score'

cfg['dataloaders']['train_batch_size'] = 4
cfg['dataloaders']['val_batch_size'] = 8

cfg['training']['optimizer'] = 'Adam'
cfg['training']['optimizer_args'] = {'lr': 0.0001}
cfg['training']['lr_scheduler'] = 'StepLR'
cfg['training']['lr_scheduler_args'] = {'step_size': 3, 'gamma': 0.1}
cfg['training']['n_epochs'] = 15
cfg['training']['early_stopping'] = False
cfg['training']['early_stopping_args'] = {'min_diff': 0.0001, 'max_epoch': 3}
cfg['training']['log_iters'] = 100
cfg['training']['best_metric'] = 'AP_IoU_0.50_all'
cfg['training']['max_iters_per_epoch'] = None

cfg['experiment_name'] = 'd_resnet50_06'

cfg_path = str(thispath.parent.parent/'calc-det/deep_learning/detection_models/config.yml')
with open(cfg_path, 'w') as yaml_file:
    yaml.dump(cfg, yaml_file, default_flow_style=False)
main()

/content/drive/MyDrive/calcification_detection/calc-det/deep_learning/dataset/dataset.py:52: DtypeWarning: Columns (16,17) have mixed types.Specify dtype option on import or set low_memory=False.
  return_lesions_mask=get_lesion_bboxes, max_lesion_diam_mm=None, use_muscle_mask=False
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

INFO:root:Storing experiment in: /content/drive/MyDrive/calcification_detection/data/deepl_runs/d_resnet50_06


creating index...
index created!
Epoch: [0]  [   0/1093]  eta: 0:33:56  lr: 0.000000  loss: 1.4958 (1.4958)  loss_classifier: 0.7567 (0.7567)  loss_box_reg: 0.0007 (0.0007)  loss_objectness: 0.6784 (0.6784)  loss_rpn_box_reg: 0.0599 (0.0599)  time: 1.8632  data: 0.3046  max mem: 6949
Epoch: [0]  [ 100/1093]  eta: 0:11:30  lr: 0.000010  loss: 0.1310 (0.3953)  loss_classifier: 0.0403 (0.1393)  loss_box_reg: 0.0076 (0.0057)  loss_objectness: 0.0567 (0.2100)  loss_rpn_box_reg: 0.0198 (0.0401)  time: 0.6827  data: 0.0096  max mem: 8837
Epoch: [0]  [ 200/1093]  eta: 0:10:15  lr: 0.000020  loss: 0.0798 (0.2632)  loss_classifier: 0.0244 (0.0908)  loss_box_reg: 0.0195 (0.0115)  loss_objectness: 0.0235 (0.1303)  loss_rpn_box_reg: 0.0106 (0.0306)  time: 0.6828  data: 0.0093  max mem: 8837
Epoch: [0]  [ 300/1093]  eta: 0:09:05  lr: 0.000030  loss: 0.0777 (0.2125)  loss_classifier: 0.0225 (0.0718)  loss_box_reg: 0.0249 (0.0181)  loss_objectness: 0.0173 (0.0969)  loss_rpn_box_reg: 0.0092 (0.0257)  t

INFO:root:Train Loss: 0.1160, AP_IoU_0.50_all: 0.7097AR_IoU_0.50_0.95_all_mdets_100: 0.5160119715675271


...

DONE (t=0.18s).
IoU metric: bbox
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.513
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.828
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.592
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.513
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.271
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.555
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.639
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.639
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = -1.000
Epo

INFO:root:Train Loss: 0.0375, AP_IoU_0.50_all: 0.8530AR_IoU_0.50_0.95_all_mdets_100: 0.7003367003367004


DONE (t=0.18s).
IoU metric: bbox
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.581
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.853
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.694
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.581
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.292
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.611
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.700
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.700
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = -1.000
Epo

INFO:root:Train Loss: 0.0206, AP_IoU_0.50_all: 0.8998AR_IoU_0.50_0.95_all_mdets_100: 0.7386831275720165


DONE (t=0.12s).
IoU metric: bbox
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.650
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.900
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.770
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.650
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.310
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.647
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.739
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.739
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = -1.000
Epo